<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

подключение бибилиотек

In [ ]:
insurance_data = pd.read_csv('/datasets/insurance.csv')
insurance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


подключение датасета и вывод информации о нем

In [ ]:
insurance_data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


вывод первых пяти строчек датасета

In [ ]:
def show_rows(data_frame):
    for column in data_frame.columns:
        print('Уникальные значения столбца', column)
        print(data_frame[column].unique())
    print('Количество пропусков в каждом столбце')    
    print(data_frame.isna().mean())

функция для показа уникальных значений столбца и количества пропусков в датасете

In [ ]:
show_rows(insurance_data)

Уникальные значения столбца Пол
[1 0]
Уникальные значения столбца Возраст
[41. 46. 29. 21. 28. 43. 39. 25. 36. 32. 38. 23. 40. 34. 26. 42. 27. 33.
 47. 30. 19. 31. 22. 20. 24. 18. 37. 48. 45. 44. 52. 49. 35. 56. 65. 55.
 57. 54. 50. 53. 51. 58. 59. 60. 61. 62.]
Уникальные значения столбца Зарплата
[49600. 38000. 21000. 41700. 26100. 41000. 39700. 38600. 49700. 51700.
 36600. 29300. 39500. 55000. 43700. 23300. 48900. 33200. 36900. 43500.
 36100. 26600. 48700. 40400. 38400. 34600. 34800. 36800. 42200. 46300.
 30300. 51000. 28100. 64800. 30400. 45300. 38300. 49500. 19400. 40200.
 31700. 69200. 33100. 31600. 34500. 38700. 39600. 42400. 34900. 30500.
 24200. 49900. 14300. 47000. 44800. 43800. 42700. 35400. 57200. 29600.
 37400. 48100. 33700. 61800. 39400. 15600. 52600. 37600. 52500. 32700.
 51600. 60900. 41800. 47400. 26500. 45900. 35700. 34300. 26700. 25700.
 33300. 31100. 31500. 42100. 37300. 42500. 27300. 46800. 33500. 44300.
 41600. 53900. 40100. 44600. 45000. 32000. 38200. 33000. 38500

Изменим тип данных в столбцах Возраст и Зарплата на целочисленный ввиду отсуствия дробных частей. 
Так же уменьшим обращение к памяти для остальных столбцов.

In [ ]:
insurance_data['Пол'] = pd.to_numeric(insurance_data['Пол'], downcast='integer')
insurance_data['Возраст'] = pd.to_numeric(insurance_data['Возраст'], downcast='integer')
insurance_data['Зарплата'] = pd.to_numeric(insurance_data['Зарплата'], downcast='integer')
insurance_data['Члены семьи'] = pd.to_numeric(insurance_data['Члены семьи'], downcast='integer')
insurance_data['Страховые выплаты'] = pd.to_numeric(insurance_data['Страховые выплаты'], downcast='integer')
insurance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int8   
 1   Возраст            5000 non-null   int8   
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int8   
 4   Страховые выплаты  5000 non-null   int8   
dtypes: float64(1), int8(4)
memory usage: 58.7 KB


Уменьшили датасет в памяти: 195.4 KB - 58.7 KB = 136.7 KB

In [ ]:
insurance_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Пол,5000.0,0.4990,0.500049,0.0,0.0,0.0,1.0,1.0
Возраст,5000.0,30.9528,8.440807,18.0,24.0,30.0,37.0,65.0
Зарплата,5000.0,39916.3600,9900.083569,5300.0,33300.0,40200.0,46600.0,79000.0
Члены семьи,5000.0,1.1942,1.091387,0.0,0.0,1.0,2.0,6.0
Страховые выплаты,5000.0,0.1480,0.463183,0.0,0.0,0.0,0.0,5.0


Вывод сводной информации (статистики) датафрейма, в транспонированном виде

## Вывод
Данные загружены и не имеют аномалий. Приведение типов позволит быстрее обрабатывать данные.

## Умножение матриц

Убедимся в том, что линейня регрессия из scikit-learn вычисляет значения по формуле.

In [ ]:
features = insurance_data.drop('Страховые выплаты',axis=1)
target = insurance_data['Страховые выплаты']

In [ ]:
X = np.concatenate((np.ones((features.shape[0], 1)), features), axis=1)
y = target
w = np.linalg.inv(X.T @ X) @ X.T @ y
display(w[1:])
model = LinearRegression()
model.fit(features, target)
model.coef_

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

array([ 7.92580543e-03,  3.57083050e-02, -1.70080492e-07, -1.35676623e-02])

Коэфициенты регрессии совпадают на 100%. Теперь перейдем к теоретической части.

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Вопрос:** Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится.

**Обоснование:** 

Умножим матрицу признаков $X$ на некую обратимую матрицу $P$ в формуле обучения вектора весов линейной регрессии $w$ :

$$
w' = ((XP^T XP)^{-1} (XP)^T y
$$

Проведем преобразования:

$$
((XP)^T XP)^{-1} (XP)^T y = 
$$

$$
(P^T X^T XP)^{-1} P^T X^T y =
$$

$$
P^{-1}(P^T X^T X)^{-1} P^T X^T y =
$$

$$
P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y =
$$

$$
P^{-1}(X^T X)^{-1} E X^T y =
$$

$$
P^{-1}(X^T X)^{-1} X^T y
$$



Теперь подставим $w'$ в формулу предсказания $a = Xw$, не забывая умножить $X$ из этой формулы на нашу матрицу $P$:

$$
a' = X P P^{-1}(X^T X)^{-1} X^T y
$$

И преобразуем:
$$
X P P^{-1}(X^T X)^{-1} X^T y = 
$$

$$
X (P P^{-1})(X^T X)^{-1} X^T y = 
$$

$$
X E (X^T X)^{-1} X^T y = 
$$

$$
X (X^T X)^{-1} X^T y = Xw
$$

В итоге получается, что  $a'$ = $a$, а значит предсказания и, следовательно, качество линейной регресии не изменятся.

<div class="alert alert-danger">
<b>🚫 Комментарий ревьювера:</b> К сожалению, у тебя в доказательстве допущена небольшая, но принципиальная ошибка:
    
в выражении $(X^T X)^{-1}$ нельзя раскрывать скобки, т.е. нельзя отдельно записывать $(X^T)^{-1}$ и $X^{-1}$, так как матрицы $X$ и $X^T$ прямоугольные и для них не существует обратных матриц. Поправь пожалуйста доказательство, не раскрывая скобки.
    
Ничего страшного, многие на этом месте ошибаются =)
</div>

## Алгоритм преобразования

**Алгоритм**

Для защиты информации на этапе поиска коэффициентов регрессии и предсказаний регрессии будем умножать матрицу признаков на обратимую матрицу Y, которая будет генерироваться случайным образом.

Этапы алгоритма:

* Составление матрицы Y.
* Проверка матрицы на обратимость. Вычисление детерминанта матрицы Y.
* Получение матрицы преобразованных признаков Z=XY.
* Применение алгоритма на преобразованных признаках Z.

**Обоснование**

Матрица Y должна иметь необходимую размерность (nxn), где n - количество признаков для регрессии. Таким образом матрица Z будет иметь туже размерность, что и матрица X. Обратная матрица Y существует только для квадратных невырожденных матриц (определитель которых не равен нулю).

Пример:
$$
X = \begin{pmatrix}
1 & 2 \\
2 & 3 \\
4 & 5  
\end{pmatrix}
\qquad 
Y = \begin{pmatrix}
1 & 0 \\
2 & 3 \\  
\end{pmatrix}
\qquad det  Y = 3
$$

Найдем значние Z:
$$
Z = \begin{pmatrix}
1 & 2 \\
2 & 3 \\
4 & 5  
\end{pmatrix} \begin{pmatrix}
1 & 0 \\
2 & 3 \\  
\end{pmatrix} = \begin{pmatrix}
1*1+2*2 & 1*0+2*3  \\
2*1+3*2 & 2*0+3*3  \\
4*1+5*2 & 4*0+5*3   
\end{pmatrix} = \begin{pmatrix}
5 & 6 \\
8 & 9 \\
14 & 15  
\end{pmatrix}
$$

После этого добавляем нулевой столбец и вводим данные в линейную регрессию.

## Проверка алгоритма

Проведем исследование модели по двум направлениям:

1. Исследуем качество модели без преобразования.
   1. С исходными признаками
   1. С отмасштабированными признаками
1. Исследуем качество модели с преобразованием.
   1. С исходными признаками
   1. С отмасштабированными признаками

Разделим данные на обучающие и тестовые.

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=13052022)

In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_origin_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef", model.coef_)
print("R2 =", R2_LR_origin_data)

w-vector coef [ 2.58872893e-03  3.55692860e-02  8.59931118e-08 -1.08074003e-02]
R2 = 0.4304021179581724


In [ ]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller), ("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_origin_data_scaled = r2_score(target_test, pipeline.predict(features_test))

print("R2 =", R2_LR_origin_data_scaled)

R2 = 0.4304021179581752


Модель ведет себя одинаково как на исходных данных, так и на отмасштабированных. Колебания 15 знака и дальше после запятой связано с точностью хранения дробных чисел в python 3.

Создадим функцию преобразования матрицы признаков.

In [ ]:
def cipher_features(features):
    crypted_features = features
    n = features.shape[1]
    np.random.seed(13052022)
    cipher_matrix = np.random.randint(1, 10, (n,n))
    det = np.linalg.det(cipher_matrix)
    while det == 0:
        np.random.seed(13052022)
        cipher_matrix = np.random.randint(1, 10, (n,n))
        det = np.linalg.det(cipher_matrix)
    crypted_features = crypted_features @ cipher_matrix
    return crypted_features, cipher_matrix

Выведем данные до преобразования и после.

In [ ]:
display(features.head())
features, cipher_matrix = cipher_features(features)
display(features.head())
cipher_matrix

,Пол,Возраст,Зарплата,Члены семьи
0,1,41,49600.0,1
1,0,46,38000.0,1
2,0,29,21000.0,0
3,0,21,41700.0,2
4,1,28,26100.0,0


,0,1,2,3
0,99503.0,99375.0,347498.0,446447.0
1,76330.0,76193.0,266324.0,342050.0
2,42203.0,42116.0,147203.0,189029.0
3,83563.0,83502.0,292051.0,375329.0
4,52404.0,52314.0,182905.0,234930.0


array([[8, 2, 9, 2],
       [7, 4, 7, 1],
       [2, 2, 7, 9],
       [8, 9, 2, 4]])

Как видно, данные потеряли свои оригинальные значения, теперь в них хранится непонятное число. Разобьем данные на тренировочную и обучающую выборку и проверим R2 значение.

In [ ]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=13052022)

In [ ]:
model = LinearRegression()
model.fit(features_train, target_train)
R2_LR_cipher_data = r2_score(target_test, model.predict(features_test))
print("w-vector coef",model.coef_)
print("R2 =", R2_LR_cipher_data)

w-vector coef [-0.01839821  0.01691878  0.01548528 -0.01171534]
R2 = 0.43040211795804917


In [ ]:
regressor = LinearRegression()
scaller = StandardScaler()
pipeline = Pipeline([("standard_scaller", scaller),("linear_regression", regressor)])
pipeline.fit(features_train, target_train)
R2_LR_cipher_data_scaled = r2_score(target_test, pipeline.predict(features_test))
print("R2 =", R2_LR_cipher_data_scaled)

R2 = 0.4304021179580899


Сравним показатели качества моделей.

In [ ]:
result = pd.DataFrame(data= [R2_LR_origin_data_scaled,
                      R2_LR_origin_data,
                      R2_LR_cipher_data,
                      R2_LR_cipher_data_scaled], 
                     columns=['R2'], 
                     index=['Линейная регрессия',
                            'Линейная регрессия c масштабом',
                            'Линейная регрессия на преобразованных признаках',
                            'Линейная регрессия на преобразованных признаках c масштабом',])
result

,R2
Линейная регрессия,0.430402
Линейная регрессия c масштабом,0.430402
Линейная регрессия на преобразованных признаках,0.430402
Линейная регрессия на преобразованных признаках c масштабом,0.430402


# Итоговый вывод
В ходе работы было проделано:

* Загружены и изучены данные.
* Качество линейной регресии не изменилось от использования исодной матрици и исходной матрицы, умноженную на обратимую.
* Создан алгоритм преобразования данных.
* Исследован алгоритм преобразования данных и проверена метрика R2 для данных без преобразования и с ним.

По результатам работы использования матричных операций можно увидеть, что данные очень просто зашифровать от распознования, имея правильную матрицу.